<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-14 15:18:14
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.21 L
Current:  1.26 C
-------------------
Today PnL: 1.03 L (0.83%)
Current PnL: -18.09 L (-12.83%)
CY Booked + Current PnL: -15.23 L (-10.8%)
-------------------
Total profit:  3.71 L
Total loss:  -21.81 L
-------------------
Total Booked + Current PnL: 11.75 L (10.31%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.29%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.06 C
Est FTT PnL: 80.59 L (64.2%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.42 C (113.21%)
Deployed:  1.14 C
Current:  1.26 C
CAGR/XIRR %: 6.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,14.94,X-MC,91.58,141085.0,26307.0,9424.0,0.47,22.92,6.68,31.13,220.0,2.79,1.14,37.43,AR,ATH,TYRES
56,PIDILITIND,2504.06,3576.00,-15.90,X-LC,46.32,93231.0,18109.0,14050.0,0.23,24.11,15.07,42.81,120.0,1.29,0.76,16.92,X40,BTT,CHEMICALS
69,SBILIFE,1496.49,1924.99,-0.75,H-LC,82.11,147370.0,21665.0,14324.0,0.83,17.23,9.72,28.63,147.0,1.51,1.19,31.29,AR,ATH,INSURANCE
62,RELIANCE,1288.53,1526.00,4.61,X-LC,90.00,224897.0,21309.0,16215.0,0.54,10.47,7.21,18.43,157.0,1.31,1.82,22.51,XY25,NTT,REFINERIES
70,SBIN,760.30,863.00,-13.45,M-LC,50.53,212026.0,10546.0,16665.0,-0.21,5.23,7.86,13.51,60.0,0.63,1.72,16.16,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.00,-19.31,H-SC,33.68,141636.0,-50379.0,129257.0,8.04,-26.24,91.26,41.08,95.0,-0.39,1.15,19.88,XY24,NTT,ENGINEERING
65,SAIL,130.64,228.00,86.16,M-MC,67.37,211617.0,-13345.0,180996.0,5.94,-5.93,85.53,74.53,221.0,-0.07,1.72,22.87,XY24,BTT,STEEL
27,FINEORG,4771.46,7168.71,-19.59,H-SC,45.26,76740.0,-4375.0,45131.0,5.02,-5.39,58.81,50.24,142.0,-0.10,0.62,28.86,X40N,ATH,CHEMICALS
75,SONACOMS,578.05,1006.00,-28.41,M-MC,88.42,92505.0,-8654.0,83541.0,4.02,-8.55,90.31,74.03,49.0,-0.10,0.75,30.00,AR,BTT,AUTO
76,STARHEALTH,551.56,761.00,-10.22,M-SC,55.79,146670.0,-53546.0,129568.0,3.76,-26.74,88.34,37.97,62.0,-0.41,1.19,17.76,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,2311.97,4911.36,-53.52,H-SC,61.05,100602.0,-1125.0,115501.0,-2.72,-1.11,114.81,112.43,2.0,-0.01,0.82,18.73,AR,ATH,MISC
91,VIPIND,488.80,718.00,-791.15,H-SC,97.89,65852.0,-29464.0,74156.0,-2.72,-30.91,112.61,46.89,82.0,-0.40,0.53,29.88,X40N,NTT,MISC
7,ASIANPAINT,2961.56,4250.00,-21.95,X-LC,5.26,155224.0,-46162.0,133772.0,-1.80,-22.92,86.18,43.51,118.0,-0.35,1.26,7.06,X40,BTT,PAINTS
78,SYMPHONY,1311.84,2037.54,-20.26,M-SC,80.00,149270.0,-1592.0,85054.0,-1.76,-1.06,56.98,55.32,15.0,-0.02,1.21,31.82,X40N,ATH,DURABLES
20,COFFEEDAY,59.14,80.00,-61.77,L-SC,48.42,55776.0,-57773.0,97826.0,-1.49,-50.88,175.39,35.27,166.0,-0.59,0.45,36.51,XR,NTT,HOTELS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-53.87,M-LC,83.16,185288.0,-20357.0,96943.0,-1.20,-9.90,52.32,37.24,1.0,-0.21,1.50,20.60,XY24,NTT,AUTO
37,INDIAMART,2311.97,4911.36,-53.52,H-SC,61.05,100602.0,-1125.0,115501.0,-2.72,-1.11,114.81,112.43,2.0,-0.01,0.82,18.73,AR,ATH,MISC
2,ABB,5319.37,8953.12,-38.51,H-LC,64.21,140725.0,7741.0,83098.0,-0.13,5.82,59.05,68.31,4.0,0.09,1.14,14.75,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-15.19,X-LC,71.58,191516.0,-20612.0,78368.0,0.90,-9.72,40.92,27.23,5.0,-0.26,1.55,9.75,X40,BTT,IT
48,LICI,777.74,983.00,-16.72,H-LC,51.58,184240.0,10026.0,35945.0,-0.47,5.76,19.51,26.39,6.0,0.28,1.49,12.79,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,14.94,X-MC,91.58,141085.0,26307.0,9424.0,0.47,22.92,6.68,31.13,220.0,2.79,1.14,37.43,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,7.09,X-MC,13.68,188769.0,33334.0,37112.0,0.02,21.45,19.66,45.32,201.0,0.90,1.53,28.44,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-15.90,X-LC,46.32,93231.0,18109.0,14050.0,0.23,24.11,15.07,42.81,120.0,1.29,0.76,16.92,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-2.94,M-LC,85.26,187745.0,36539.0,29119.0,0.09,24.17,15.51,43.43,37.0,1.25,1.52,30.14,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LUXIND,1448.19,2442.00,33.16,M-SC,26.32,138320.0,742.0,93670.0,2.03,0.54,67.72,68.62,79.0,0.01,1.12,19.91,X40N,NTT,TEXTILES
57,POLYCAB,5005.92,7541.35,6.97,H-MC,87.37,167370.0,27204.0,43784.0,-0.50,19.41,26.16,50.65,55.0,0.62,1.36,27.74,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,1.45,H-SC,74.21,165672.0,14308.0,31478.0,2.46,9.45,19.00,30.25,111.0,0.45,1.34,31.12,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-2.94,M-LC,85.26,187745.0,36539.0,29119.0,0.09,24.17,15.51,43.43,37.0,1.25,1.52,30.14,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-5.43,H-SC,38.95,139604.0,1744.0,36018.0,3.20,1.26,25.80,27.39,68.0,0.05,1.13,23.85,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WIPRO,243.46,420.00,-8.35,M-LC,41.05,156711.0,5766.0,103696.0,0.47,3.82,66.17,72.51,52.0,0.06,1.27,17.99,XR,NTT,IT
18,CAMS,3643.00,5250.99,-9.11,H-SC,8.42,102200.0,196.0,44825.0,0.66,0.19,43.86,44.14,22.0,0.00,0.83,16.46,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,34.74,129561.0,409.0,86663.0,1.15,0.32,66.89,67.42,41.0,0.00,1.05,26.01,XR,NTT,BANKS
37,INDIAMART,2311.97,4911.36,-53.52,H-SC,61.05,100602.0,-1125.0,115501.0,-2.72,-1.11,114.81,112.43,2.0,-0.01,0.82,18.73,AR,ATH,MISC
92,VOLTAS,1272.00,1929.20,-31.16,H-MC,12.63,137511.0,-2409.0,74696.0,-0.60,-1.72,54.32,51.67,32.0,-0.03,1.12,3.86,AR,ATH,AC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,40.02,M-SC,84.21,121446.0,-21033.0,40551.0,2.30,-14.76,33.39,13.70,214.0,-0.52,0.98,43.34,X40N,NTT,BANKS
53,MRF,114777.60,150504.12,14.94,X-MC,91.58,141085.0,26307.0,9424.0,0.47,22.92,6.68,31.13,220.0,2.79,1.14,37.43,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-0.75,H-LC,82.11,147370.0,21665.0,14324.0,0.83,17.23,9.72,28.63,147.0,1.51,1.19,31.29,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-12.58,M-LC,75.79,153450.0,12610.0,31887.0,3.20,8.95,20.78,31.59,48.0,0.40,1.24,33.29,XY24,NTT,BANKS
14,BERGEPAINT,558.54,680.00,-18.97,H-MC,47.37,173985.0,-3072.0,41582.0,1.10,-1.74,23.90,21.75,154.0,-0.07,1.41,24.46,XY24,NTT,PAINTS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,QUESS,711.66,986.00,NaN,NaN,100.00,80789.0,-75065.0,135144.0,1.51,-48.16,167.28,38.55,NaN,-0.56,0.66,23.71,XY24,NTT,MISC
80,TATAELXSI,7332.28,9000.00,-18.67,X-MC,98.95,85379.0,-17273.0,40623.0,1.88,-16.83,47.58,22.74,47.0,-0.43,0.69,28.33,X40N,NTT,IT
91,VIPIND,488.80,718.00,-791.15,H-SC,97.89,65852.0,-29464.0,74156.0,-2.72,-30.91,112.61,46.89,82.0,-0.40,0.53,29.88,X40N,NTT,MISC
79,TANLA,917.30,1963.11,-39.41,H-SC,96.84,148065.0,-91350.0,364299.0,1.79,-38.16,246.04,114.01,20.0,-0.25,1.20,35.80,AR,ATH,IT
1,AARTIIND,487.04,919.00,35.16,M-SC,95.26,77605.0,-3731.0,75867.0,-0.43,-4.59,97.76,88.69,159.0,-0.05,0.63,30.86,XR,NTT,CHEMICALS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.51
1,25,39.20
2,50,68.31


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.03
X40N    15.28
X40     14.51
AR      14.11
XR      10.36
XY25     8.38
X200     1.23
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.51
M-SC    17.90
X-LC    16.44
M-LC     8.41
H-MC     7.85
X-MC     6.78
H-LC     5.91
M-MC     5.60
X-SC     2.82
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.79,-2.05,34.51
BANKS,11.19,-9.16,62.70
IT,9.35,-20.05,87.02
FINANCE,7.49,-19.78,73.84
MISC,7.20,-27.62,82.54
INSURANCE,6.10,-2.54,33.45
PAINTS,5.56,-19.98,55.49
ELECTRICAL,4.13,-9.20,64.84
HEALTHCARE,3.65,-3.50,36.91


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3032061.0,27
X40N,1280675.0,18
XR,1266930.0,15
AR,1263739.0,15
X40,804850.0,12
SR,208195.0,2
XY25,135374.0,5
X200,66792.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2412867.0,22
M-SC,2034797.0,21
X-LC,714716.0,12
M-MC,577048.0,5
H-MC,428933.0,7
X-SC,361006.0,4
M-LC,342181.0,6
L-SC,318330.0,4
X-MC,318189.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      920792.0      6
M-SC       XY24      766232.0      6
H-SC       X40N      553704.0      7
           AR        540931.0      3
M-SC       X40N      456764.0      6
           XR        430882.0      5
M-MC       XY24      406844.0      3
X-LC       X40       376892.0      5
H-SC       XR        302115.0      4
M-LC       XY24      221820.0      4
H-LC       AR        207215.0      4
M-SC       X40       200565.0      2
X-LC       AR        188268.0      3
H-MC       XY24      180332.0      2
L-SC       XR        165027.0      2
X-MC       XY24      140807.0      2
X-SC       SR        122047.0      1
           XR        120461.0      1
           X40N      118498.0      2
X-MC       X40N      107925.0      2
M-LC       XR        103696.0      1
H-SC       X40        95325.0      2
M-SC       AR         94206.0      1
L-SC       XY24       87845.0      1
M-MC       XR         86663.0      1
M-SC       SR         86148.0      1
M-MC       AR         83541.0      1
X-LC       XY25       82764.0      3
H-MC       AR         74696.0      1
           X40        72035.0      2
X-LC       X200       66792.0      1
L-SC       AR         65458.0      1
X-MC       X40        60033.0      1
H-MC       XR         58086.0      1
           X40N       43784.0      1
H-LC       XY25       35945.0      1
M-LC       XY25       16665.0      1
X-MC       AR          9424.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
